## BeatGAN: Anomalous Rhythm Detection using Adversarially Generated Time

### Abstract
Given a large-scale rhythmic time series containing mostly normal data segments (or ‘beats’), can we learn how to detect anomalous beats in an effective yet efficient way? For example, how can we detect anomalous beats from electrocardiogram (ECG) readings? Existing approaches either require excessively high amounts of labeled and balanced data for classification, or rely on less regularized reconstructions, resulting in lower accuracy in anomaly detection. Therefore, we propose BeatGAN, an unsupervised anomaly detection algorithm for time series data. BeatGAN outputs explainable results to pinpoint the anomalous time ticks of an input beat, by comparing them to adversarially generated beats. Its robustness is guaranteed by its regularization of reconstruction error using an adversarial generation approach, as well as data augmentation using time series warping. Experiments show that BeatGAN accurately and efficiently detects anomalous beats in ECG time series, and routes doctors’ attention to anomalous time ticks, achieving accuracy of nearly 0.95 AUC, 
and very fast inference (2.6 ms per beat). In addition, we show that BeatGAN accurately detects unusual motions from multivariate motion-capture time series data, illustrating its generality.

You can configure the backend to use GPU or CPU only. \
Default is using backend cpu. 

In [ ]:
import spartan as st

```loadTensor``` function automatically read data from file and ```toDTensor``` function extract time and value separately from the tensor.<br/>```Timeseries``` class is designed to construct time tensor.

In [ ]:
time, value = st.loadTensor(path = "inputData/ecg_timeseries_small.zip", col_types = [float, float], sep=' ').toDTensor(hastticks=True)
time_series = st.Timeseries(value, time)

### Run RPeak to segment ECG series data.

This example use ECG series data as example. Before we train the model, we need RPeak to preprocess data.<br/>
Default parameters are:<br/>
{'sampling_rate': 360, 'left_size': 120, 'right_size': 136, 'out_path': None}

In [ ]:
rpeak = st.RPeak(time_series)
segments = rpeak.run()

In [ ]:
segments.shape

### Run BeatGAN as a single model

BeatGAN has two kinds of structure, CNN and RNN which is controled by parameter ```network```.

Default parameters are: <br/>
{'seq_len': 256, 'stride': 32, 'input_size': 1, 'rep_size': 20, 'batch_size': 64, 'max_epoch':5, 'lr': 0.01, 'lambda': 1
'net_type':'gru', 'layers':1, 'hidden_size':100} <br/>
You can change them be passing <key>=<value> as the doc shows.

In [ ]:
beatgan_CNN = st.BeatGAN(segments, "my_beatgan_model", network="CNN")

In [ ]:
trained_model = beatgan_CNN.fit()

### Use PipeLine to finish the work mentioned above.

In [ ]:
rpeak_params = {'sampling_rate': 360, 'left_size': 120, 'right_size': 136}
beatgan_params = {'network': 'CNN', 'seq_len': 256, 'stride': 32, 'input_size': 1, 'rep_size': 20, 'model_name': 'my_beatgan_model'}
pipeline = st.PipeLine(time_series, [(st.RPeak, rpeak_params), ((st.Train, st.TrainPolicy.BeatGAN), beatgan_params)])

We will get a trained model after calling ```run``` function of ```pipeline```.

In [ ]:
pipeline.run()

### Save and load trained models

Function ```save_model_to``` saves model to assigned path.

In [ ]:
trained_model.save_model_to(path='./trained_model/test_cnn.pth')

Parameter ```model_path``` loads model from assigned path.

In [ ]:
beatgan_CNN = st.BeatGAN(segments, "my_beatgan_model", network="CNN", model_path="./trained_model/ecg_cnn.pth")
rec_diff, ori_ts, rec_ts, rec_err = beatgan_CNN.predict()

Sort by max window reconstruction error and get the sorted indices.

In [ ]:
import numpy as np
new_score=[]
window_size=32
for scores in rec_err:
    tmp_score=0
    for i in range(0,len(scores),window_size):
        tmp_score=max(tmp_score, np.mean(scores[i:i+window_size]))
    new_score.append(tmp_score)
ind=np.argsort(new_score)

Draw plots of segments which deviate the least and the most with ```st.plot```.

In [ ]:
st.plot(st.BeatGAN, ori_ts[ind[0]], rec_ts[ind[0]], rec_err)
st.plot(st.BeatGAN, ori_ts[ind[-1]], rec_ts[ind[-1]], rec_err)

Deviate the least | Deviate the most
:-------------------------:|:-------------------------:
<img src="images/beatganRec1.png" width="300"/>  |   <img src="images/beatganRec2.png" width="300"/>
<b>Segment with maximum reconstruction error. |  <b>Segment with minimum reconstruction error.

### Run BeatGAN from train task to get a trained model

In [ ]:
train_task = st.Train.create(segments, st.TrainPolicy.BeatGAN, 'my_beatgan_model', network='CNN')

In [ ]:
train_task.run()

### Experiment Results
------
BeatGAN Structure      |  BeatGAN on Motion Datasets
:-------------------------:|:-------------------------:
<img src="images/beatganExp1.png" width="300"/>  |   <img src="images/beatganExp2.png" width="300"/>
<b>Structure of BeatGAN. |  <b>Beatlex reconstructs motion data.

|BeatGAN's fast inference|
|:-------------------------:|
|<img src="images/beatganExp3.png" width="300"/>|
|<b>BeatGAN has fast inference.|

### Cite:
------
1. Zhou, B., Liu, S., Hooi, B., Cheng, X., & Ye, J. (2019, August). BeatGAN: Anomalous Rhythm Detection using Adversarially Generated Time Series. In IJCAI (pp. 4433-4439).

    <details>
    <summary><span style="color:blue">click for BibTex...</span></summary>

    ```bibtex
    @inproceedings{zhou2019beatgan,
      title={BeatGAN: Anomalous Rhythm Detection using Adversarially Generated Time Series.},
      author={Zhou, Bin and Liu, Shenghua and Hooi, Bryan and Cheng, Xueqi and Ye, Jing},
      booktitle={IJCAI},
      pages={4433--4439},
      year={2019}
    }
    ```
    </details>